In [2]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import torch
import numpy as np
%matplotlib inline
import re 
import math
import time
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [4]:
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.metrics.pairwise import cosine_similarity
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
from gensim.models import KeyedVectors




In [0]:
RANDOM_SEED = 123
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [7]:
DEVICE

device(type='cuda', index=0)

In [8]:
import os
import pandas as pd

# read the text file and add the column names
# read_file = pd.read_csv(r"./booksummaries.txt", sep='	', header=None)  Colab Notebooks/
new_file = pd.read_csv(r"/content/drive/My Drive/final project/movies_genres_en.csv", sep='\t', encoding='utf-8')
new_file = new_file.iloc[:1300, :]
new_file.head()

,title,plot,Action,Adult,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,Game-Show,History,Horror,Music,Musical,Mystery,News,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western,plot_lang
0,"""#7DaysLater"" (2013)",#7dayslater is an interactive comedy series f...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,en
1,"""#BlackLove"" (2015) {Crash the Party (#1.9)}","With just one week left in the workshops, the...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,en
2,"""#BlackLove"" (2015) {Making Lemonade Out of Le...",All of the women start making strides towards...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,en
3,"""#BlackLove"" (2015) {Miss Independent (#1.5)}",All five of these women are independent and s...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,en
4,"""#BlackLove"" (2015) {Sealing the Deal (#1.10)}",Despite having gone through a life changing p...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,en


In [0]:
# import os
# import pandas as pd

# # read the text file and add the column names
# # read_file = pd.read_csv(r"./booksummaries.txt", sep='	', header=None)  Colab Notebooks/
# read_file = pd.read_csv(r"/content/drive/My Drive/Colab Notebooks/final project/booksummaries.txt", sep='	', header=None)
# read_file.columns = ['ID', 'm number', 'book name', 'author name', 'date', 'label', 'summary']

# # clean data
# read_file['label'] = read_file['label'].str.replace(r'/m/\S*\s', '')
# read_file['label'] = read_file['label'].str.replace(r'{', '')
# read_file['label'] = read_file['label'].str.replace(r'}', '')
# read_file['label'] = read_file['label'].str.replace(r'novel', 'fiction', case = False)
# read_file['label'] = read_file['label'].str.replace(r'\\u00e0\s+clef', '')

# # select columns
# new_file = read_file.loc[:, ['book name', 'label', 'summary']]

# #delete the columns with no labels
# new_file.dropna(axis = 0, how = 'any', inplace = True)
# new_file = new_file.iloc[:, [0, 2, 1]]

# new_file = new_file.reset_index(drop=True)

# #output data as csv
# #new_file.to_csv(r'/content/drive/My Drive/Colab Notebooks/final project/booksummries.csv', index=False)


In [9]:
from sklearn.utils import shuffle
new_file = shuffle(new_file)
new_file.head().reset_index(drop=True)

,title,plot,Action,Adult,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,Game-Show,History,Horror,Music,Musical,Mystery,News,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western,plot_lang
0,"""60 Minutes"" (1968) {Heart Attack!/The Kids fr...","""Heart Attack!"" Is a segment of 60 Minutes pr...",0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,en
1,"""2point4 Children"" (1991) {Misery (#2.8)}","Whilst Rona spends Christmas alone,all her ex...",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,en
2,"""60 Minutes"" (1968) {Blackwater 61/Fight for a...","""Blackwater 61"" rebroadcasts an investigation...",0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,en
3,"""17 Kids and Counting"" (2008) {Duggars Under W...",The Duggars are under water taking a trip to ...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,en
4,"""48 Hours"" (1988) {Pain Killers (#12.26)}",CBS News 48 Hours takes a look at pain. For m...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,en


  ## Encoding the Labels##




In [10]:
one_hot = new_file.iloc[:, 2:-1]
one_hot.head()

,Action,Adult,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,Game-Show,History,Horror,Music,Musical,Mystery,News,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western
1111,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
624,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1020,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
319,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
892,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [11]:
one_hot.shape

(1300, 27)

## Split the words##

In [12]:



#def words_process(new_file):
book_summaries = new_file['plot']
summary_list = [summary for summary in book_summaries]
summary_num = len(summary_list)
#summaries = ''.join(summary_list)
print("the total number of books: {}\n".format(summary_num))

all_docs = []
i_index = 0
for doc in summary_list:
    # Tokenize the string into words
    tokens = word_tokenize(doc)
    # Remove non-alphabetic tokens, such as punctuation
    words = [word.lower() for word in tokens if word.isalpha()]
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if not word in stop_words]
    if len(words) >= 5:
      all_docs.append(words)
    else:
      print(i_index)
      one_hot = one_hot.drop(i_index, axis=0)
      print(one_hot.shape)
    i_index += 1
    #return all_docs, one_hot_delt

one_hot = one_hot.reset_index(drop=True)

the total number of books: 1300



In [0]:
# l1 = []
# for i in range(len(all_docs)):
#   l1.append(len(all_docs[i]))
# #print(l1.index(6))
# len_counts = Counter(l1)
# df_len = pd.DataFrame(len_counts.items(), columns=['len', '#books'])
# df_len = df_len.sort_values(by = 'len', ascending = True)
# #print(min(l1))
# df_len.head(17)

# #one_hot = one_hot.reset_index(drop=True)

## outliners ##

## Using a Pre-Trained Embedding Layer

In [0]:
# import multiprocessing
# import sys
# import gensim
# from gensim.models import Word2Vec
# assert gensim.models.doc2vec.FAST_VERSION > -1, "This will be painfully slow otherwise."
# #from UtilWordEmbedding import TfidfEmbeddingVectorizer
# from sklearn.feature_extraction.text import TfidfVectorizer
# workers = multiprocessing.cpu_count()
# print('number of cpu: {}'.format(workers))

# word_model = Word2Vec(all_docs,
#                       min_count=1,
#                       size=300,
#                       window=5,
#                       workers=workers,
#                       iter=100)

In [15]:

# def words_embedding(docs):
   
all_words = [ word for doc in all_docs for word in doc]
all_words_nodup = list(dict.fromkeys(all_words))
# Load word2vec model (trained on an enormous Google corpus)
# google_vecs = KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin', binary = True) /Colab Notebooks
google_vecs = KeyedVectors.load_word2vec_format('/content/drive/My Drive/final project/GoogleNews-vectors-negative300.bin', binary = True)
embedding_dim_dim = google_vecs.vector_size

def words_embedding(doc):
    # Filter the list of vectors to include only those that Word2Vec has a vector for
    vector_list = [google_vecs[word] for word in doc if word in google_vecs.vocab]
    #vector_list = [word_model.wv[word] for word in doc]
    #google_vectors = np.asarray(vector_list)
    # Create a list of the words corresponding to these vectors
    words_filtered = [word for word in doc if word in google_vecs.vocab]
    #Zip the words together with their vector representations
    word_vec_zip = zip(words_filtered, vector_list)

    # Cast to a dict so we can turn it into a DataFrame
    word_vec_dict = dict(word_vec_zip)
    word_vec = pd.DataFrame.from_dict(word_vec_dict, orient='index')
    return word_vec

word_vec  = words_embedding(all_words_nodup)
word_vec_array = np.array(word_vec)
word_vec.shape

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


(10680, 300)

In [16]:

len(all_words)
#word_vec_array.shape
word_vec_array

array([[-0.15039062,  0.24316406,  0.06396484, ...,  0.140625  ,
        -0.0612793 ,  0.28320312],
       [ 0.18066406,  0.13183594,  0.07714844, ..., -0.33007812,
        -0.24414062,  0.1328125 ],
       [-0.140625  ,  0.12011719, -0.19238281, ...,  0.03637695,
         0.03417969, -0.32226562],
       ...,
       [-0.36523438, -0.15332031,  0.01843262, ..., -0.11132812,
         0.19628906, -0.2578125 ],
       [-0.22265625,  0.109375  ,  0.19238281, ..., -0.06640625,
         0.05639648,  0.20703125],
       [-0.00267029,  0.27148438, -0.03369141, ..., -0.03564453,
         0.01599121, -0.13183594]])

In [0]:
word_vec_array = np.insert(word_vec_array, 0, np.zeros(word_vec_array.shape[1]), 0)


In [18]:
word_vec.tail()


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
foresees,0.152344,0.068359,0.075684,-0.116211,0.002350,-0.019409,0.247070,-0.169922,-0.053711,-0.033447,-0.330078,-0.079102,0.023315,0.024780,-0.096680,0.263672,-0.090332,0.410156,0.025513,-0.008850,-0.371094,0.376953,-0.029297,-0.029053,0.075195,-0.190430,-0.056396,0.015991,-0.231445,0.244141,-0.075195,-0.203125,-0.103516,0.417969,0.113770,-0.179688,-0.129883,0.172852,0.085938,-0.028442,...,-0.096680,0.121094,0.205078,0.009460,0.210938,0.239258,-0.176758,-0.174805,-0.215820,0.140625,0.099121,-0.234375,-0.072754,-0.296875,-0.116211,0.083008,-0.101562,-0.108887,0.300781,-0.045654,-0.147461,0.166992,-0.353516,-0.113770,0.308594,0.160156,-0.003693,0.042969,0.192383,-0.036377,-0.447266,-0.181641,-0.104980,-0.189453,-0.116699,0.081055,-0.099609,0.093262,0.324219,-0.023560
instigate,0.022217,0.098145,0.152344,-0.224609,-0.269531,0.149414,0.013062,0.055420,0.045410,-0.088379,-0.206055,-0.185547,-0.018066,0.376953,0.051514,0.054199,-0.162109,0.012268,0.147461,0.000185,0.178711,0.191406,0.247070,0.022095,-0.020752,0.124512,-0.161133,0.106934,0.037354,0.150391,-0.117188,0.021606,0.025024,-0.082031,-0.275391,0.013428,-0.029785,0.154297,0.208984,0.243164,...,-0.041504,0.045654,-0.047852,0.153320,-0.051270,-0.143555,-0.218750,0.052246,0.095215,-0.170898,-0.053955,0.163086,0.104492,0.296875,0.015320,0.035156,-0.041260,0.026001,0.216797,-0.068359,0.088379,0.026855,0.098145,-0.094238,0.067383,-0.014404,-0.211914,-0.061768,0.091797,-0.212891,-0.090820,0.197266,0.083984,0.186523,-0.174805,-0.361328,0.052246,0.005341,0.138672,0.141602
champagne,-0.365234,-0.153320,0.018433,0.233398,-0.013062,-0.150391,0.033203,-0.287109,0.215820,0.255859,-0.207031,-0.378906,0.257812,-0.081055,-0.143555,0.376953,0.165039,0.102539,0.296875,-0.253906,0.141602,0.361328,0.138672,-0.462891,0.134766,0.449219,-0.376953,0.302734,0.109863,0.220703,-0.279297,-0.199219,0.184570,-0.007263,-0.012451,-0.021606,0.088867,-0.292969,-0.044434,-0.246094,...,-0.357422,0.283203,-0.237305,0.201172,-0.010254,-0.316406,-0.050781,0.009583,-0.007324,-0.013489,-0.003387,0.180664,0.122559,0.092285,0.302734,0.121582,-0.017090,0.126953,0.013855,-0.218750,-0.043457,-0.041992,0.184570,-0.148438,0.292969,0.298828,0.079590,0.394531,0.121582,-0.032715,0.261719,-0.137695,0.043945,-0.032959,0.020508,-0.357422,0.296875,-0.111328,0.196289,-0.257812
tupperware,-0.222656,0.109375,0.192383,0.589844,-0.022339,0.242188,0.143555,-0.090332,-0.267578,0.127930,0.021973,-0.238281,0.058838,0.059814,0.036865,0.076172,-0.040039,0.072266,-0.051025,-0.093750,0.183594,-0.042236,0.013184,0.071289,0.063965,0.144531,-0.165039,0.243164,-0.109863,-0.277344,-0.117676,0.102051,-0.050537,0.072266,0.151367,-0.009888,0.215820,-0.004883,-0.104004,-0.099609,...,0.081543,0.078125,-0.253906,-0.281250,-0.070801,0.055908,-0.289062,-0.009338,-0.199219,0.028442,0.018921,0.144531,0.341797,0.216797,0.168945,-0.005585,0.061768,-0.083008,-0.167969,0.032471,-0.092773,0.076660,0.069824,0.168945,0.152344,-0.034180,0.064453,0.032227,-0.214844,0.064453,-0.088379,0.028809,-0.100098,0.113770,0.028687,-0.197266,-0.025513,-0.066406,0.056396,0.207031
sacrifices,-0.002670,0.271484,-0.033691,0.464844,-0.006622,-0.199219,0.302734,-0.158203,0.163086,0.294922,-0.090820,-0.040771,-0.144531,0.402344,-0.002090,-0.010437,0.005676,0.084473,-0.047119,-0.188477,-0.026611,0.104980,0.048340,-0.223633,0.187500,0.215820,-0.175781,-0.386719,-0.180664,-0.375000,-0.113281,-0.021484,0.443359,0.136719,0.196289,-0.361328,0.425781,0.298828,-0.094727,0.052979,...,0.115234,0.037598,-0.074219,0.099609,0.244141,0.165039,0.189453,0.074219,-0.281250,0.078125,-0.081543,0.277344,0.472656,0.101074,0.211914,-0.066895,-0.086426,0.229492,-0.486328,0.049072,0.259766,-0.047363,0.032471,-0.060303,-0.292969,0.006531,

In [0]:
# word_vec_array

In [0]:
# word_vec.to_csv(r'./wor_vec.csv', index=False)

In [0]:
#word_vec = pd.read_csv(r"/content/drive/My Drive/Colab Notebooks/final project/wor_vec.csv", header=None)

# Generate the integer vectors of all summaries

In [0]:
word_index = pd.DataFrame(index = word_vec.index)
wordlen_list = range(1, word_vec.shape[0]+1)
word_index['index'] = wordlen_list


In [23]:
wordlen_list[-1]

10680

In [0]:

def words_index(all_docs):
    all_words_index = []
    for doc in all_docs:
        inds = []
        for word in doc:
            try:
                inds.append(word_index.at[word, 'index'])
            except: 
                continue
            
        all_words_index.append(inds)
    return all_words_index

all_words_index = words_index(all_docs)
# all_words_index[-1]

In [25]:
len(all_words_index)

1300

In [26]:
max(all_words_index[-1])

10680

In [27]:
l = []
for i in range(len(all_words_index)):
    l.append(len(all_words_index[i]))
l.index(max(l))
max(l)
#len(all_words_index)

174

# Visualize the cluster of books

In [0]:
# import numpy as np
# def avg_doc_embedding(all_docs):
#     doc_vec = []
#     for doc in all_docs:
#         if len(doc) != 0:
#             vector =  words_embedding(doc)
#             doc_vec.append(np.mean(np.array(vector), axis=0))
    
#     summary_vec = np.array(doc_vec)
#     return summary_vec

# summary_vec = avg_doc_embedding(all_docs)

In [0]:
# summary_vec.shape

In [30]:
pip install gensim

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
#def tf_dif(doc):
all_text_docs = []
for doc in all_docs:
    # print(len(doc))
    all_text_docs.append(" ". join(doc))
tf_idf_vect = TfidfVectorizer(stop_words='english', max_features=6000)
final_tf_idf = tf_idf_vect.fit_transform(all_text_docs)
tfidf_feat = tf_idf_vect.get_feature_names()
    #return final_tf_idf
# final_tf_idf
# tfidf_feat

In [32]:
tfidf_doc_vectors_ft = [] # the tfidf-ft for each summary  is stored in this list
row=0
errors=0
for doc in all_docs: # for each summary 
    doc_vec = np.zeros(300) # as word vectors are of zero length
    weight_sum =0; # num of words with a valid vector in the summary 
    for word in doc: # for each word in a summary 
        try:
            word_vec = google_vecs[word]
            #word_vec = word_model.wv[word]
            # obtain the tf_idf of a word in a summary 
            tfidf = final_tf_idf[row, tfidf_feat.index(word)]
            #print(tfidf)
            doc_vec += (word_vec * tfidf)
            weight_sum += tfidf
        except:
            errors =+1
            pass
    doc_vec /= weight_sum
   

    tfidf_doc_vectors_ft.append(doc_vec)
    row += 1

tfidf_doc_vectors_ft_array = np.array(tfidf_doc_vectors_ft)
print('errors noted: '+str(errors))

errors noted: 1


In [0]:
# tfidf_doc_vectors_ft_array.shape

In [34]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

X = tfidf_doc_vectors_ft_array
y = one_hot.values
print(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train, y_train)


[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [35]:
print(X_test.shape)
y_pred = neigh.predict(X_test)

print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))


(325, 300)
Testing accuracy 0.5353846153846153
Testing F1 score: 0.6980464729371313


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)


In [0]:
# pip install adjustText

In [0]:
# from sklearn.manifold import TSNE
# from adjustText import adjust_text
# import seaborn as sns
# import matplotlib.pyplot as plt
# book_titles = new_file['title']
# titles_list = [title for title in book_titles]
# titles_list = titles_list[:400]
# def t_SNE_plot(df_data):
#     # Initialize t-SNE
#     tsne = TSNE(n_components = 2, init = 'random', random_state = 10, perplexity = 100)

#     # Use only 400 rows to shorten processing time
#     tsne_df = tsne.fit_transform(df_data)#tsne.fit_transform(df_data[:400])

#     sns.set()
#     # Initialize figure
#     fig, ax = plt.subplots(figsize = (11.7, 8.27))
#     sns.scatterplot(tsne_df[:, 0], tsne_df[:, 1], alpha = 0.5)

#     # Import adjustText, initialize list of texts

#     texts = []
#     data_to_plot = list(np.arange(0, 400, 20))#list(np.arange(0, 400, 40))

#     # Append words to list
#     for data in data_to_plot:
#         texts.append(plt.text(tsne_df[data, 0], tsne_df[data, 1], titles_list[data], fontsize = 14))

#     # Plot text using adjust_text (because overlapping text is hard to read)
#     adjust_text(texts, force_points = 0.4, force_text = 0.4, 
#                 expand_points = (2,1), expand_text = (1,2),
#                 arrowprops = dict(arrowstyle = "-", color = 'black', lw = 0.5))

#     plt.show()

#     return tsne_df

# tsne_df = t_SNE_plot(tfidf_doc_vectors_ft_array[:400, :])    

In [0]:
# # An user defined function to create scatter plot of vectors
# def scatter(x, colors):
#     # We choose a color palette with seaborn.
#     palette = np.array(sns.color_palette("hls", 27))

#     # We create a scatter plot.
#     f = plt.figure(figsize=(32, 32))
#     ax = plt.subplot(aspect='equal')
#     sc = ax.scatter(x[:,0], x[:,1], lw=0, s=120,
#                     c=palette[colors.astype(np.int)])
#     #plt.xlim(-25, 25)
#     #plt.ylim(-25, 25)
#     ax.axis('off')
#     ax.axis('tight')

#     # We add the labels for each cluster.
#     txts = []
#     for i in range(27):
#         # Position of each label.
#         xtext, ytext = np.median(x[colors == i, :], axis=0)
#         txt = ax.text(xtext, ytext, str(i), fontsize=50)
#         txt.set_path_effects([
#             PathEffects.Stroke(linewidth=5, foreground="w"),
#             PathEffects.Normal()])
#         txts.append(txt)

#     return f, ax, sc, txts

# Y = one_hot.iloc[:400, :].to_numpy()
# print(list(range(0,18)))
# sns.palplot(np.array(sns.color_palette("hls", 27)))
# scatter(tsne_df, Y)   

## Padding sequences

In [39]:
max_Length = 0  
docs_length = []
for x in all_words_index:
    docs_length.append(len(x))
    if len(x) > max_Length:
        max_Length = len(x)

docs_length = np.array(docs_length)
max_Length

174

In [40]:
np.where(docs_length<1)
#docs_length[3382]

(array([], dtype=int64),)

In [0]:
def pad_sequences(all_words_index, seq_length):
    
    # getting the correct rows x cols shape
    sequences = np.zeros((len(all_words_index), seq_length), dtype=int)
 
    # for each review, I grab that review and 
    for i, row in enumerate(all_words_index):
        if len(row)>0:
            sequences[i, 0:len(row)] =  np.array(row)[:seq_length]
    
    return sequences

In [42]:
sequences = pad_sequences(all_words_index, max_Length)

assert len(sequences)==len(all_words_index), "Sequences should have as many rows as reviews."
assert len(sequences[0])==max_Length, "Each sequence row should contain seq_length values."

print(sequences[-1])
np.where(sequences[-1] ==51242 )

[  282  3342  3343  6049   251  1358   496   880  3188  1179  2094   281
   239  3258  6049  1087   275  3945   719  2766   282   292  1358  6049
  3307   781  7316  2301    58   254   281    43 10680   101   108   107
  3085   263  1028  2774   271   263  1205  5219  2774     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0   

(array([], dtype=int64),)

# Training, Validation, and Test Data

In [43]:
split_frac = 0.7

## split data into training, validation, and test data (features and labels, x and y)

split_idx = int(len(sequences)*split_frac)
train_x, remaining_x = sequences[:split_idx], sequences[split_idx:]
train_y, remaining_y = one_hot[:split_idx], one_hot[split_idx:]
train_len, remaining_len = docs_length[:split_idx], docs_length[split_idx:]

# train_x, remaining_x = sequences[:10000], sequences[-4000:]
# train_y, remaining_y = one_hot[:10000], one_hot[-4000:]
# train_len, remaining_len = docs_length[:10000], docs_length[-4000:]

test_idx = int(len(remaining_x)*0.5)
val_x, test_x = remaining_x[:test_idx], remaining_x[test_idx:]
val_y, test_y = remaining_y[:test_idx], remaining_y[test_idx:]
val_len, test_len = remaining_len[:test_idx], remaining_len[test_idx:]

## print out the shapes of your resultant feature data
print("\t\t\tSequences Shapes:")
print("Train set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))


			Sequences Shapes:
Train set: 		(909, 174) 
Validation set: 	(195, 174) 
Test set: 		(196, 174)


In [44]:
test_len.min

<function ndarray.min>

In [0]:
# split_frac = 0.8

# ## split data into training, validation, and test data (features and labels, x and y)

# split_idx = int(len(all_words_index)*split_frac)
# train_x, remaining_x = all_words_index[:split_idx], all_words_index[split_idx:]
# train_y, remaining_y = one_hot[:split_idx], one_hot[split_idx:]
# #train_len, remaining_len = docs_length[:split_idx], docs_length[split_idx:]

# test_idx = int(len(remaining_x)*0.5)
# val_x, test_x = remaining_x[:test_idx], remaining_x[test_idx:]
# val_y, test_y = remaining_y[:test_idx], remaining_y[test_idx:]
# #val_len, test_len = remaining_len[:test_idx], remaining_len[test_idx:]

# ## print out the shapes of your resultant feature data
# print("\t\t\tSequences Shapes:")
# print("Train set: \t\t{}".format(len(train_x)), 
#       "\nValidation set: \t{}".format(len(val_x)),
#       "\nTest set: \t\t{}".format(len(test_)))

### Above only 2,0,1 because I only used  3 datasample to do all of these

# DataLoaders and Batching

In [0]:
import torch
from torch.utils.data import TensorDataset, DataLoader, Dataset
# from torch.nn.utils.rnn import pad_sequence
# # create Tensor datasets


train_data = TensorDataset(torch.from_numpy(np.array(train_x)), torch.from_numpy(np.array(train_y).astype(np.int16)), torch.from_numpy(train_len)) 
valid_data = TensorDataset(torch.from_numpy(np.array(val_x)), torch.from_numpy(np.array(val_y).astype(np.int16)), torch.from_numpy(val_len))
test_data = TensorDataset(torch.from_numpy(np.array(test_x)), torch.from_numpy(np.array(test_y).astype(np.int16)), torch.from_numpy(test_len)) 

# # train_data = Dataset(train_x, train_y) #, torch.from_numpy(train_len)
# # valid_data = Dataset(val_x, val_y)
# # test_data = Dataset(test_x, test_y) 

# # dataloaders
batch_size = 128
# #RANDOM_SEED = 1

# def pad_collate(batch):
#   (xx, yy) = zip(*batch)
#   x_lens = [len(x) for x in xx]
#   y_lens = [len(y) for y in yy]

#   xx_pad = pad_sequence(xx, batch_first=True, padding_value=0)
#   yy_pad = pad_sequence(yy, batch_first=True, padding_value=0) , collate_fn=pad_collate

#   return xx_pad, yy_pad, x_lens, y_lens

# shuffling and batching data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size, num_workers=4)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size, num_workers=4)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size, num_workers=4)

In [47]:
len(train_loader)

8

# LSTM model

In [0]:


# import numpy as np
# from torch import nn
# import torch.nn.functional as F


# # def kmax_pooling(x, dim, k):
# #     index = x.topk(k, dim = dim)[1].sort(dim = dim)[0]
# #     return x.gather(dim, index)

# class LSTMText(torch.nn.Module): 
#     def __init__(self, vocab_size, embedding_dim, hidden_size, num_layers, 
#                 linear_hidden_size, num_classes, freeze_embeddings=True):
#         super(LSTMText, self).__init__()
        
#         self.num_classes = num_classes
#         # 1. embedding layer
#         self.embedding = nn.Embedding(vocab_size, embedding_dim)
#         # set weights to pre-trained
#         self.embedding.weight = nn.Parameter(torch.from_numpy(word_vec_array)) # all vectors
#         # (optional) freeze embedding weights
#         if freeze_embeddings:
#             self.embedding.requires_grad = False

#         self.lstm =nn.LSTM( input_size = embedding_dim,
#                             hidden_size = hidden_size,
#                             num_layers = num_layers,
#                             bias = True,
#                             batch_first = False,
#                             # dropout = 0.5,
#                             bidirectional = True
#                             )

#         # self.dropout = nn.Dropout()
#         self.fc = nn.Sequential(
#             nn.Linear(hidden_size*2, linear_hidden_size),
#             nn.BatchNorm1d(linear_hidden_size),
#             nn.ReLU(inplace=True),
#             nn.Linear(linear_hidden_size, num_classes)
#         )
        
#          #activation function
#         self.act = nn.Sigmoid()
 
#     def forward(self, text):
      
#         #text = [batch size,sent_length]
#         embedded = self.embedding(text)
#         #embedded = [batch size, sent_len, emb dim]
#         print(embedded.shape)
#         #packed sequence
#         #packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths, batch_first=True)
        
#         #packed_output, (hidden, cell) = self.lstm(packed_embedded)
#         packed_output = self.lstm(embedded) #.view(len(text), 1, -1)
#         #hidden = [batch size, num layers * num directions,hid dim]
#         #cell = [batch size, num layers * num directions,hid dim]
        
#         #concat the final forward and backward hidden state
#         #hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)
                
#         #hidden = [batch size, hid dim * num directions]
#         #logits=self.fc(hidden)
#         logits=self.fc(packed_output) #.view(len(text),-1)

#         #Final activation function
#         probas=self.act(logits, dim = 1)

#         return logits, probas


In [0]:
import numpy as np
from torch import nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
# attention layer code inspired from: https://discuss.pytorch.org/t/self-attention-on-words-and-masking/5671/4
class Attention(nn.Module):
    def __init__(self, hidden_size, batch_first=False):
        super(Attention, self).__init__()

        self.hidden_size = hidden_size
        self.batch_first = batch_first

        self.att_weights = nn.Parameter(torch.Tensor(1, hidden_size), requires_grad=True)

        stdv = 1.0 / np.sqrt(self.hidden_size)
        for weight in self.att_weights:
            nn.init.uniform_(weight, -stdv, stdv)

    def get_mask(self):
        pass

    def forward(self, inputs, lengths):
        if self.batch_first:
            batch_size, max_len = inputs.size()[:2]
        else:
            max_len, batch_size = inputs.size()[:2]
        # print(inputs.size())
        # print(self.att_weights.permute(1, 0).unsqueeze(0).repeat(batch_size, 1, 1).size())    
        # apply attention layer
        weights = torch.bmm(inputs,
                            self.att_weights  # (1, hidden_size)
                            .permute(1, 0)  # (hidden_size, 1)
                            .unsqueeze(0)  # (1, hidden_size, 1)
                            .repeat(batch_size, 1, 1) # (batch_size, hidden_size, 1)
                            )
    
        attentions = torch.softmax(F.tanh(weights.squeeze()), dim=-1)

        # create mask based on the sentence lengths
        mask = torch.ones(attentions.size(), requires_grad=True).cuda()
        for i, l in enumerate(lengths):  # skip the first sentence
            if l < max_len:
                mask[i, l:] = 0

        # apply mask and renormalize attention scores (weights)
        masked = attentions * mask
        _sums = masked.sum(-1).unsqueeze(-1)  # sums per row
        
        attentions = masked.div(_sums)

        # apply attention weights
        weighted = torch.mul(inputs, attentions.unsqueeze(-1).expand_as(inputs))

        # get the final fixed vector representations of the sentences
        representations = weighted.sum(1).squeeze()

        return representations, attentions

In [0]:


import numpy as np
from torch import nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

# def kmax_pooling(x, dim, k):
#     index = x.topk(k, dim = dim)[1].sort(dim = dim)[0]
#     return x.gather(dim, index)

class LSTMText1(torch.nn.Module): 
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_layers, 
                linear_hidden_size, num_classes, freeze_embeddings=True):
        super(LSTMText1, self).__init__()
        
        
#         self.num_classes = num_classes
        # 1. embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        # set weights to pre-trained
        self.embedding.weight = nn.Parameter(torch.from_numpy(word_vec_array)) # all vectors
        # (optional) freeze embedding weights
        if freeze_embeddings:
            self.embedding.requires_grad = False

        self.lstm =nn.LSTM( input_size = embedding_dim,
                            hidden_size = hidden_size,
                            num_layers = num_layers,
                            bias = True,
                            batch_first = True,
                            dropout = 0.5,
                            bidirectional = True
                            )
        self.atten = Attention(hidden_size*2, batch_first=True) # 2 is bidrectional
        # self.lstm2 = nn.LSTM(input_size=hidden_size*2,
        #                     hidden_size=hidden_size,
        #                     num_layers=1, 
        #                     bidirectional=True)
        # self.dropout = nn.Dropout()
        self.fc = nn.Sequential(
            nn.Dropout(),
            nn.Linear(hidden_size*2, linear_hidden_size), #linear_hidden_size
            nn.BatchNorm1d(linear_hidden_size),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(linear_hidden_size, num_classes)
        )
        
         #activation function
        self.act = nn.Sigmoid()
 
    def forward(self, text, text_lengths):
      
        #text = [batch size,sent_length]
        embedded = self.embedding(text)
        #embedded = [batch size, sent_len, emb dim]
        #print(embedded.shape)
        #packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths, batch_first=True, enforce_sorted=False)
        
        packed_output, (hidden, cell) = self.lstm(packed_embedded)
        # print(packed_output.shape)
        text, text_lengths = nn.utils.rnn.pad_packed_sequence(packed_output, batch_first=True)
        #packed_output = self.lstm(embedded) #.view(len(text), 1, -1)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
        text, _ = self.atten(text, text_lengths) 
        # print('text: ', text.size())
        # out2, (h_n, c_n) = self.lstm2(packed_output)
        # y, lengths = nn.utils.rnn.pad_packed_sequence(out2, batch_first=True)
        # y, _ = self.atten(y, lengths)
        # print('y: ', y.size())

        #conv_out = kmax_pooling((content_out),2,self.opt.kmax_pooling)
        #concat the final forward and backward hidden state
        #hidden = torch.cat([x, y], dim=1)
        # hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)
                
        # hidden = [batch size, hid dim * num directions]
        logits=self.fc(text)
        # reshaped = hidden.view( hidden.size(0), -1)
        # logits=self.fc(hidden) #.view(len(text),-1)

        #Final activation function
        probas=self.act(logits)

        return logits, probas


In [0]:


import numpy as np
from torch import nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

# def kmax_pooling(x, dim, k):
#     index = x.topk(k, dim = dim)[1].sort(dim = dim)[0]
#     return x.gather(dim, index)

class TextCNN(torch.nn.Module): 
    def __init__(self, vocab_size, embedding_dim, num_filters, kernel_sizes,
                 num_classes, freeze_embeddings=True):
        super(TextCNN, self).__init__()
        
        
#         self.num_classes = num_classes
        # 1. embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        # set weights to pre-trained
        self.embedding.weight = nn.Parameter(torch.from_numpy(word_vec_array)) # all vectors
        # (optional) freeze embedding weights
        if freeze_embeddings:
            self.embedding.requires_grad = False

        # 2. convolutional layers
        self.convs = nn.ModuleList([
                nn.Sequential(nn.Conv1d(in_channels = embedding_dim, 
                                        out_channels = covhidden_dim, 
                                        kernel_size = h),
                              nn.BatchNorm1d(covhidden_dim), 
                              nn.ReLU(),
                              nn.MaxPool1d(kernel_size = max_Length-h+1))
                     for h in kernel_sizes
                    ])
        # self.convs_1d = nn.ModuleList([ nn.Sequential(                        
        #     nn.Conv1d(1, num_filters, (k, embedding_dim), padding=(k-2,0)) 
        #     for k in kernel_sizes]
        #     nn.BatchNorm1d(covhidden_dim),))
        
        # 3. final, fully-connected layer for classification
        self.fc = nn.Sequential(
                nn.Dropout(),
                nn.Linear(len(kernel_sizes) * num_filters, linear_hidden_size),
                nn.BatchNorm1d(linear_hidden_size),
                nn.Dropout(),
                nn.ReLU(inplace=True),
                nn.Linear(linear_hidden_size,num_classes)
        )

       
        self.act = nn.Sigmoid()

    def conv_and_pool(self, x, conv):
      """
      Convolutional + max pooling layer
      """
      # squeeze last dim to get size: (batch_size, num_filters, conv_seq_length)
      # conv_seq_length will be ~ 200
      x = F.relu(conv(x)).squeeze(3)
      
      # 1D pool over conv_seq_length
      # squeeze to get size: (batch_size, num_filters)
      x_max = F.max_pool1d(x, x.size(2)).squeeze(2)
      return x_max
 
    def forward(self, text):
      
      #text = [batch size,sent_length]
      embedded = self.embedding(text)
      #  creates a channel dimension that conv layers expect
      embedded = embedded.permute(0, 2, 1)
      #embedded = [batch size, sent_len, emb dim]
      # get output of each conv-pool layer
      conv_results = [conv(embedded) for conv in self.convs]
      # conv_results = [self.conv_and_pool(embedded, conv) for conv in self.convs_1d]
      
      # concatenate results and add dropout
      text = torch.cat(conv_results, dim = 1)
      text = text.view(-1, text.size(1)) 
      # final logit
      logits = self.fc(text) 

      #Final activation function
      probas=self.act(logits)

      return logits, probas


In [0]:
def kmax_pooling(x, dim, k):
    index = x.topk(k, dim = dim)[1].sort(dim = dim)[0]
    return x.gather(dim, index)

class RCNN(torch.nn.Module): 
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_layers, covhidden_dim, 
                linear_hidden_size, kernel_size, kmax_pooling,num_classes, freeze_embeddings=True ):
        super(RCNN, self).__init__()
        
        # self.kernel_size = kernel_size #[2,3,4]
        self.kmax_pooling = kmax_pooling
        # 1. embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        # set weights to pre-trained
        self.embedding.weight = nn.Parameter(torch.from_numpy(word_vec_array)) # all vectors
        # (optional) freeze embedding weights
        if freeze_embeddings:
            self.embedding.requires_grad = False

        self.lstm =nn.LSTM( input_size = embedding_dim,
                            hidden_size = hidden_size,
                            num_layers = num_layers,
                            bias = True,
                            batch_first = True,
                            dropout = 0.5,
                            bidirectional = True
                            )
        
        self.conv = nn.Sequential(
            nn.Conv1d(in_channels = hidden_size*2 ,
                      out_channels = covhidden_dim,
                      kernel_size =  kernel_size),
            nn.BatchNorm1d(covhidden_dim),
            nn.ReLU(inplace=True),

            nn.Conv1d(in_channels = covhidden_dim,
                      out_channels = covhidden_dim,
                      kernel_size =  kernel_size),
            nn.BatchNorm1d(covhidden_dim),
            nn.ReLU(inplace=True),

            
            nn.MaxPool1d(kernel_size)
        )
        # self.dropout = nn.Dropout()
        self.fc = nn.Sequential(
            nn.Dropout(),
            nn.Linear(self.kmax_pooling*covhidden_dim, linear_hidden_size),
            nn.BatchNorm1d(linear_hidden_size),
            nn.Dropout(),
            nn.ReLU(inplace=True),
            nn.Linear(linear_hidden_size,num_classes)
        )
        # self.fc = nn.Linear(3 * (opt.title_dim+opt.content_dim), opt.num_classes)
       
 
    def forward(self, text, text_lengths):
        
        embedded = self.embedding(text)
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths, batch_first=True, enforce_sorted=False)
        
        packed_output, (hidden, cell) = self.lstm(packed_embedded)
        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)
        

        # content_out = self.content_lstm(content.permute(1,0,2))[0].permute(1,2,0)
        # content_em = (content).permute(0,2,1)
        # content_out = t.cat((hidden,content_em),dim=1)


        conv_out = self.conv(hidden) #kmax_pooling(self.conv(hidden),2,self.kmax_pooling)

        
        reshaped = conv_out.view(conv_out.size(0), -1)
        logits = self.fc(reshaped)
        probas=self.act(logits)
        return logits, probas

# Instantiate the network

In [64]:
# vocab_size, embedding_dim, hidden_size, num_layers, 
#                  bidirectional, dropout, linear_hidden_size, num_classes
    
# Instantiate the model with hyperparameters
import random
vocab_size = word_vec.shape[0]+1#len(google_vecs.vocab)
#num_classes = 1 # binary class (1 or 0)
embedding_dim = google_vecs.vector_size # 300-dim vectors
hidden_size = 256 #LSTM hidden size
num_layers=2 #LSTM layers
linear_hidden_size = 200 # units number of full connected 
num_classes = one_hot.shape[1]
num_filters = 100
kernel_sizes = [3, 4, 5]
kmax_pooling = 2
covhidden_dim = 128

# random.seed(RANDOM_SEED)
# torch.manual_seed(RANDOM_SEED)  ###revise


# torch.nn.init.xavier_uniform(m.weight.data)

model = LSTMText1(vocab_size, embedding_dim, hidden_size, num_layers, 
                 linear_hidden_size, num_classes)

# model = TextCNN(vocab_size, embedding_dim, num_filters, kernel_sizes,
#                  num_classes)
# model = model.float()

print(model)
model = model.to(DEVICE)

#No. of trianable parameters
def count_parameters(model):
    return sum(m.numel() for m in model.parameters() if m.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')


LSTMText1(
  (embedding): Embedding(301, 300)
  (lstm): LSTM(300, 256, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  (atten): Attention()
  (fc): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=512, out_features=200, bias=True)
    (2): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): ReLU(inplace=True)
    (4): Dropout(p=0.5, inplace=False)
    (5): Linear(in_features=200, out_features=27, bias=True)
  )
  (act): Sigmoid()
)
The model has 6,032,983 trainable parameters


In [0]:
NUM_EPOCHS = 50
intial_lr = 0.3
criterion = nn.BCELoss()       
optimizer = torch.optim.SGD(model.parameters(), lr=intial_lr, weight_decay=4e-4, momentum=0.9) #Adam 
# scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer,
#                                                    gamma=0.1,
#                                                    last_epoch=-1)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 10, gamma=0.5, last_epoch=-1)
#scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.01, max_lr=0.1)

In [60]:
  for epoch in range(30):
    if not epoch % 10 and epoch != 0:
      print(epoch)
probas = np.array([[0.25, 0.4, 0.6, 0.9, 0.12], [0.25, 0.4, 0.6, 0.9, 0.12], [0.25, 0.4, 0.6, 0.9, 0.12], [0.25, 0.4, 0.6, 0.9, 0.12]]  ) 
#(probas > 0.4).cpu().numpy().astype(np.uint8)   
target_index = np.argsort(probas[1, :])#[-3:]  
np.argsort(probas[1, :])[-3:]

10
20


array([1, 2, 3])

# Define loss function and evaluation parameters and Train

In [0]:
def compute_accuracy_and_loss(model, data_loader, device):
    correct_pred, num_examples = 0, 0
    loss = 0.
    accuracy_list = []
    f_measure_list = []
    list_prob = []
    for i, (features, targets, features_len) in enumerate(data_loader):
            
        features = features.to(device)
        targets = targets.to(device)
        features_len = features_len.to(device)
        # print('features_len', features_len.shape)
        # print('targets', targets.shape)
        # print('features', features.shape)
        logits, probas = model(features, features_len)
        #list_prob.append(probas.cpu().numpy().tolist())
        loss += criterion(probas, targets.float()).item()
        # targets_arr = targets.cpu().numpy()
        # # print(targets_arr)
        # targets_num = np.count_nonzero(targets_arr, axis=1)
        # # print(targets_num)
        # probas_arr = probas.cpu().numpy() 
        # predicted_labels = np.zeros(probas_arr.shape)
        # for i in range(probas_arr.shape[0]):
        #   target_index = np.argsort(probas_arr[i, :])[-targets_num[i]:]
        #   for j in target_index:
        #     predicted_labels[i, j] = 1
        predicted_labels = (probas > 0.6).cpu().numpy().astype(np.uint8)#(probas > 0.4).cpu().numpy()
       
        print('target:', targets[:20])
        print('probas:', probas[:20])
        #print(predicted_labels)
        # print(precision_recall_fscore_support(targets.cpu().numpy(), predicted_labels,average='weighted')[:3])
        num_examples += targets.size(0)
        accuracy_list.append(accuracy_score(targets.cpu().numpy(), predicted_labels))
        f_measure_list.append(list(precision_recall_fscore_support(targets.cpu().numpy(), predicted_labels,average='weighted')[:3]))
    average_loss = loss/ num_examples
    # accuracy = accuracy_score(targets.cpu().numpy(), predicted_labels)
    # #ACU = roc_auc_score(targets.cpu().numpy(), predicted_labels)
    # f_measure = precision_recall_fscore_support(targets.cpu().numpy(), predicted_labels,average='weighted')
    accuracy = np.mean(np.array(accuracy_list))
    f_measure = np.mean(np.array(f_measure_list), axis = 0)
    return accuracy, average_loss, f_measure

In [66]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_fscore_support
import torch.nn.functional as F


    

start_time = time.time()
train_acc_lst, valid_acc_lst = [], []
train_loss_lst, valid_loss_lst = [], []

for epoch in range(NUM_EPOCHS):
    
    model.train()
    
    for batch_idx, (features, targets, features_len) in enumerate(train_loader): #, 

        ### PREPARE MINIBATCH
        features = features.to(DEVICE)
        targets = targets.to(DEVICE)
        features_len = features_len.to(DEVICE)
#         features, features_lengths = batch.features
#         targets = batch.targets
        #print(features_len)
        #retrieve text and no. of words
        #print(features_len)
            
        ### FORWARD AND BACK PROP
        features = features.view(features.size(0), -1)
        logits, probas = model(features, features_len)
        cost = criterion(probas, targets.float())
        optimizer.zero_grad()
        
        cost.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        # model.apply(weights_init)
        ### LOGGING
        if not batch_idx % 20:
            print (f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d} | '
                   f'Batch {batch_idx:03d}/{len(train_loader):03d} |' 
                   f' Cost: {cost:.4f}')
        if not epoch % 15 and epoch != 0:
            scheduler.step()
        
    # no need to build the computation graph for backprop when computing accuracy
    model.eval()
    with torch.set_grad_enabled(False):
        train_acc, train_loss, train_f_measure = compute_accuracy_and_loss(model, train_loader, device=DEVICE)
        valid_acc, valid_loss, valid_f_measure = compute_accuracy_and_loss(model, valid_loader, device=DEVICE)
        train_acc_lst.append(train_acc)
        valid_acc_lst.append(valid_acc)
        train_loss_lst.append(train_loss)
        valid_loss_lst.append(valid_loss)
        print(f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d} Train Acc.: {train_acc:.2f}'
              f' | Validation Acc.: {valid_acc:.2f}')

    elapsed = (time.time() - start_time)/60
    print(f'Time elapsed: {elapsed:.2f} min')
    print('the precision_recall_fscore_support of F-score_train: ', train_f_measure)
    print('the precision_recall_fscore_support of F-score_val: ', valid_f_measure)




RuntimeError: ignored

In [0]:
plt.plot(range(1, NUM_EPOCHS+1), train_loss_lst, label='Training loss')
plt.plot(range(1, NUM_EPOCHS+1), valid_loss_lst, label='Validation loss')
plt.legend(loc='upper right')
plt.ylabel('Binary cross entropy')
plt.xlabel('Epoch')
plt.show()

In [0]:
plt.plot(range(1, NUM_EPOCHS+1), train_acc_lst, label='Training accuracy')
plt.plot(range(1, NUM_EPOCHS+1), valid_acc_lst, label='Validation accuracy')
plt.legend(loc='upper left')
plt.ylabel('accuracy')
plt.xlabel('Epoch')
plt.show()
print(f'Training accuracy: {train_acc_lst[-1]:.2f}%')
print(f'Validation accuracy: {valid_acc_lst[-1]:.2f}%')

In [0]:

model.eval()
with torch.set_grad_enabled(False): # save memory during inference
    test_acc, test_loss, test_f_measure = compute_accuracy_and_loss(model, test_loader, DEVICE)
    print(f'Test accuracy: {test_acc:.2f}%')

In [0]:
for i, (features, targets, features_len) in enumerate(test_loader):
            
#         features = features.to(device)
#         targets = targets.to(device)
#         features_len = features_len.to(device)
        print('features_len', features_len.shape)
        print('targets', targets.shape)
        print('features', features.shape)